# Testing flat_field step with MIRI simulated data

## Summary

This notebook processes an image through calwebb_image2 and calwebb_image3 (calwebb_detector1 is optional) and examines the output table of the source_catalog step. The steps are as follow:

1) Set up data path and directory and image file name.

2) Modify header information of input simulations (if needed).

3) Run input data through calwebb_detector1.

4) Run output of calwebb_detector1 through the flat_field step in calwebb_image2.

5) Get flat field reference file. 

6) Compare the flat field reference file with the rate/cal image ratio and check that they are the same.


The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

Author: T. Temim

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

### Set up import statements

In [2]:
import pytest
import numpy as np
from glob import glob
import json
import jwst
from astropy.io import fits, ascii
from astropy.coordinates import Angle
from astropy.table import Table, vstack, unique
from astropy.stats import sigma_clip
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline
from jwst.associations import asn_from_list
import matplotlib.pyplot as plt
import random
from jwst import associations
from jwst.datamodels import RampModel
from astropy.io import fits
import numpy as np
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.pipeline import calwebb_image3
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_detector1
from astropy.io import fits
from jwst.datamodels import ImageModel
from jwst import datamodels
from astropy.utils.data import get_pkg_data_filename
from ci_watson.artifactory_helpers import get_bigdata
from astropy import table
import crds

In [3]:
import os
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_CONTEXT']='jwst_0619.pmap'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

### Print pipeline version number

In [4]:
jwst.__version__ 

'1.1.0'

### Read in data from artifactory

In [5]:
input_file = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'source_catalog',
                     'source_catalog_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits')

### Read in input image as JWST data model

In [6]:
from jwst import datamodels
im = ImageModel(input_file)

### Modify header information of input simulations (if needed)

In [7]:
print(im.meta.wcsinfo.wcsaxes)

4


In [8]:
im.meta.wcsinfo.wcsaxes=2

In [9]:
print(im.meta.wcsinfo.wcsaxes)

2


In [10]:
del im.meta.wcsinfo.cdelt3
del im.meta.wcsinfo.crpix3
del im.meta.wcsinfo.crval3
del im.meta.wcsinfo.ctype3
del im.meta.wcsinfo.cunit3
del im.meta.wcsinfo.pc3_1
del im.meta.wcsinfo.pc3_2
#del im.meta.wcsinfo.cdelt4
#del im.meta.wcsinfo.crpix4
#del im.meta.wcsinfo.crval4
#del im.meta.wcsinfo.ctype4
#del im.meta.wcsinfo.cunit4

### Run input data through calwebb_detector1  (not done here)

In [11]:
#det1 = calwebb_detector1.Detector1Pipeline()
#det1.save_results = True
#det1.run(im)

### Run output of calwebb_detector1 through calwebb_image2

In [12]:
input_file = input_file.replace('rateint.fits', 'rate.fits')

In [13]:
im2 = calwebb_image2.Image2Pipeline()
#im2.background.skip = True
im2.assign_wcs.skip = True
im2.flat_field.skip = False
im2.photom.skip=True
im2.resample.skip = True
im2.save_results = True
im2.run(im)

2021-03-16 20:31:25,823 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2021-03-16 20:31:25,825 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2021-03-16 20:31:25,826 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2021-03-16 20:31:25,827 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.


2021-03-16 20:31:25,828 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.


...

2021-03-16 20:31:28,380 - stpipe.Image2Pipeline - INFO - Finished processing product det_image_seq1_MIRIMAGE_F560Wexp1_rate


2021-03-16 20:31:28,381 - stpipe.Image2Pipeline - INFO - ... ending calwebb_image2


2021-03-16 20:31:28,441 - stpipe.Image2Pipeline - INFO - Saved model in det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits


2021-03-16 20:31:28,442 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline done


[<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits>]

In [14]:
input_file = input_file.replace('rate.fits', 'cal.fits')

In [15]:
im_cal = ImageModel(input_file)

### Calculate the rate/cal image ratio

In [16]:
ratio_im=im.data/im_cal.data

2021-03-16 20:31:28,483 - stpipe - WARNING - <ipython-input-1-13a01c86caeb>:1: RuntimeWarning: invalid value encountered in true_divide
  ratio_im=im.data/im_cal.data



### Get flat_field reference file

In [17]:
flatreffile = im_cal.meta.ref_file.flat.name
print('Flat reference file', flatreffile)
# find location of file
basename = crds.core.config.pop_crds_uri(flatreffile)
path = crds.locate_file(basename, "jwst")
# open reference file
flat_im = ImageModel(path)

Flat reference file crds://jwst_miri_flat_0507.fits


### Compare flat field reference file with the rate/cal image ratio and check that they are equal

In [18]:
check_flat=ratio_im/flat_im.data

In [19]:
print('Minimum and maximum values in the check_flat image are:', np.nanmin(check_flat), 'and', np.nanmax(check_flat))

Minimum and maximum values in the check_flat image are: 0.9999999 and 1.0000001


### Both values above should equal to 1.0